In [142]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mpl_toolkits
%matplotlib inline

data = pd.read_csv('example.csv', ';',encoding='latin1')
# data = pd.read_csv('~/Dropbox/workspace-python/LZDataPrepAndML/outV5.csv', '\t',encoding='latin1')
# data.count()


In [143]:
print(data.head())

   EID  CID                Time Event                  Response
0    1    1   12-05-2017 09:53     va                   Warning
1    2    1   13-05-2017 13:35     po  DistractClient;Seclusion
2    3    1   26-05-2017 09:32     va                   Warning
3    4    1   26-05-2017 11:02     pp            DistractClient
4    5    2   21-06-2017 14:51     va            DistractClient


In [144]:
columnTime = "Time"
columnAction = "Event"
columnResponse = "Response"
columnCaseid = "CID"
columnEventid = "EID"

# columnTime = "Algemeen: Datum incident"
# columnAction = "Aggression_short"
# columnResponse = "[B09] Maatregelen om agressie te stoppen"
# columnCaseid = "Pseudoniem_client"
# columnEventid = "Meldingsnummer"



columnEffect = "Next_Type_Eps"

In [145]:


data[columnTime] = pd.to_datetime(data[columnTime])
# data = data.sort_values('Algemeen: Datum incident') # This now sorts in date 

##
## Sorting the incident. Only then the shift works well. 
##
data.sort_values([columnCaseid, columnTime, columnEventid], ascending=[True, True, True], inplace=True)


In [146]:
#
# Shift the data that is grouped by the clients to calculate the next incident type and next incident date
#
data['Next_Type'] = data.groupby(columnCaseid)[columnAction].shift(-1)
data['Next_DayOfIncident'] = data.groupby(columnCaseid)[columnTime].shift(-1)

#
# Set the data type of the next date 
#
data['Next_DayOfIncident'] = pd.to_datetime(data['Next_DayOfIncident'])

#
# Calculate the difference. 
#
data["Next_DaysToNext"] = (data['Next_DayOfIncident'] - data[columnTime])/np.timedelta64(1,'D')

In [147]:
### Step 1 : preprocess log using epsilon
eps = 7

data.loc[data['Next_DaysToNext'] <= eps, columnEffect] = data['Next_Type']
data.loc[data['Next_DaysToNext'] > eps, columnEffect] = 'Tau' 

# data.fillna("NaN", inplace=True)

In [148]:

data[columnEffect].describe()

count       7
unique      4
top       Tau
freq        3
Name: Next_Type_Eps, dtype: object

In [123]:
cNextT = columnAction
cCurrT = columnEffect

setActions = pd.Series(data[columnAction]).drop_duplicates().tolist()
print(setActions)

columnname = columnResponse
cMeasureValues = pd.Series(data[columnResponse]).drop_duplicates().tolist()
# print(cMeasureValues)

setResponse = set()
for i in cMeasureValues:
    for v in str(i).split(';'):
        setResponse.add(v.strip())

print(setResponse)


['va', 'sib', 'pp', 'po']
{'client toegesproken/gesprek met client', 'geen', 'met kracht tegen- of vastgehouden', 'naar andere kamer/ruimte gestuurd', 'afzondering (deur op slot)', 'contact beeindigd/weggegaan', 'client afgeleid', 'preventieve maatregelen gestart', 'nan'}


In [135]:

data['Next_Type_Eps'].replace(np.nan, "NaN", inplace = True)


# FF = calcFreqMeasuresCombined2DDict(data, columnname+cMeasureValues[0])
FF = calcFreqMeasures2DDict(data, columnAction, columnResponse, columnEffect, setResponse)
# F = calcFreq2DDict(data, cMeasureT)




label = "va"
F = FF[label]
print(F)

# for ai in F:
#     F[ai][ai] = 0


{'columnTot': {'Tau': 2996, 'va': 1336, 'sib': 112, 'pp': 786, 'NaN': 417, 'po': 131}, 'client toegesproken/gesprek met client': {'Tau': 2218, 'pp': 567, 'va': 958, 'NaN': 301, 'sib': 85, 'po': 93}, 'contact beeindigd/weggegaan': {'Tau': 1066, 'sib': 31, 'pp': 293, 'va': 484, 'NaN': 128, 'po': 39}, 'nan': {'va': 76, 'Tau': 164, 'NaN': 26, 'pp': 40, 'sib': 10, 'po': 4}, 'geen': {'Tau': 134, 'va': 93, 'NaN': 15, 'pp': 40, 'sib': 10, 'po': 8}, 'naar andere kamer/ruimte gestuurd': {'Tau': 738, 'va': 334, 'sib': 32, 'pp': 205, 'NaN': 72, 'po': 28}, 'client afgeleid': {'pp': 178, 'Tau': 428, 'NaN': 48, 'va': 223, 'po': 24, 'sib': 24}, 'preventieve maatregelen gestart': {'Tau': 36, 'NaN': 8, 'va': 5, 'sib': 1, 'pp': 5}, 'met kracht tegen- of vastgehouden': {'pp': 54, 'va': 46, 'Tau': 138, 'po': 12, 'NaN': 13, 'sib': 9}, 'afzondering (deur op slot)': {'pp': 55, 'sib': 2, 'Tau': 57, 'po': 9, 'va': 12, 'NaN': 1}}


In [136]:
# -----------------
# Example : two dimentional dictionary to data frame
# ------------------
# tableRe = pd.DataFrame(Frelative).T

# print(tableRe)
# chi2_cont_row(tableRe)

table = pd.DataFrame(F).T
print(table)
table.fillna(0, inplace=True)
table.drop(columns=['NaN'], inplace = True)
table.drop(index=['columnTot'], inplace = True)
table.dropna
print(table)
stat, p, dof, expected = chi2_cont_row(table)

                                          NaN     Tau     po     pp    sib  \
columnTot                               417.0  2996.0  131.0  786.0  112.0   
client toegesproken/gesprek met client  301.0  2218.0   93.0  567.0   85.0   
contact beeindigd/weggegaan             128.0  1066.0   39.0  293.0   31.0   
nan                                      26.0   164.0    4.0   40.0   10.0   
geen                                     15.0   134.0    8.0   40.0   10.0   
naar andere kamer/ruimte gestuurd        72.0   738.0   28.0  205.0   32.0   
client afgeleid                          48.0   428.0   24.0  178.0   24.0   
preventieve maatregelen gestart           8.0    36.0    NaN    5.0    1.0   
met kracht tegen- of vastgehouden        13.0   138.0   12.0   54.0    9.0   
afzondering (deur op slot)                1.0    57.0    9.0   55.0    2.0   

                                            va  
columnTot                               1336.0  
client toegesproken/gesprek met client   95

In [139]:
import graphviz
import pygraphviz as pgv

# chi-squared test with similar proportions
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.sandbox.stats.multicomp import multipletests
import statsmodels.api as sm       

from itertools import combinations 

pos = "all"

label = 'va'
start = label
name = label
F = FF[label]
print(F)

table = pd.DataFrame(F).T
table.replace(np.nan,0, inplace = True)
table.drop(columns=['NaN'], inplace = True)
print(table)

G = pgv.AGraph(strict=False, directed=True, landscape=True)
G.graph_attr['rankdir'] ='LR'
G.node_attr['shape']= 'box'
G.clear()
    

pvalues = list() # keep a list of computed p values to use for adjusting p-values

adjresidual = 2.57
for ai in F:
    if(ai == 'columnTot'):
        continue
    print("=======")
    print(ai)
    newF = dict()
    newF['non'] = dict()
    newF[ai] = dict()
    bool_observed_zero = False
    for aj in table.columns:
        # expected cell >= 5
        # observed count cell >= 1
        # observed count 75% cells  >= 5

        if(aj not in F[ai] or F[ai][aj] == 0 or pd.isnull(F[ai][aj])):
            bool_observed_zero = True
            print(ai)
            break
    
        newF[ai][aj] = F[ai][aj]
        newF['non'][aj] = table.at['columnTot',aj] - F[ai][aj]
    if bool_observed_zero:
        print("skip")
        continue
    

    newTable = pd.DataFrame(newF).T
    stat, p, dof, expected = chi2_cont_row(newTable)
    
    expectedT = pd.DataFrame(expected, columns = newTable.columns, index = newTable.index)

    
    if (p <= 0.05):
        rtable = sm.stats.Table(newTable)     
#         print(rtable.resid_pearson)
#         print(rtable.standardized_resids)
        
        subtable = pd.DataFrame(rtable.standardized_resids)
#         print(subtable)
#         print(newTable)
#         print(expectedT)
        
        for aj in table.columns:
            if (subtable.at[ai,aj] < -adjresidual) or (subtable.at[ai,aj] > adjresidual):
                if(G.has_edge(start+"_s", ai) == False):
                    rowtot = newTable.sum(axis=1)[ai]
                    G.add_edge(start+"_s", ai, label="{0:.0f}".format(rowtot))
                if(subtable.at[ai,aj] > 2.57):
                    penwidthl='3'
                else:
                    penwidthl='1'
                G.add_edge(ai, aj, penwidth=penwidthl, label="{0:.0f}".format(newTable.at[ai,aj]) + "("+ "{0:.0f}".format(expectedT.at[ai,aj]) +")")
    pvalues.append(p)
    
G.draw('./graph_' + name + '-asr' + pos + '.png', prog='dot')
    
    
p_adjusted = multipletests(pvalues, alpha=.05, method='bonferroni')
# Print the resulting conclusions
print(p_adjusted[0])

# Print the adjusted p-values themselves 
print(p_adjusted[1])
print("////////////")


{'columnTot': {'Tau': 2996, 'va': 1336, 'sib': 112, 'pp': 786, 'NaN': 417, 'po': 131}, 'client toegesproken/gesprek met client': {'Tau': 2218, 'pp': 567, 'va': 958, 'NaN': 301, 'sib': 85, 'po': 93}, 'contact beeindigd/weggegaan': {'Tau': 1066, 'sib': 31, 'pp': 293, 'va': 484, 'NaN': 128, 'po': 39}, 'nan': {'va': 76, 'Tau': 164, 'NaN': 26, 'pp': 40, 'sib': 10, 'po': 4}, 'geen': {'Tau': 134, 'va': 93, 'NaN': 15, 'pp': 40, 'sib': 10, 'po': 8}, 'naar andere kamer/ruimte gestuurd': {'Tau': 738, 'va': 334, 'sib': 32, 'pp': 205, 'NaN': 72, 'po': 28}, 'client afgeleid': {'pp': 178, 'Tau': 428, 'NaN': 48, 'va': 223, 'po': 24, 'sib': 24}, 'preventieve maatregelen gestart': {'Tau': 36, 'NaN': 8, 'va': 5, 'sib': 1, 'pp': 5}, 'met kracht tegen- of vastgehouden': {'pp': 54, 'va': 46, 'Tau': 138, 'po': 12, 'NaN': 13, 'sib': 9}, 'afzondering (deur op slot)': {'pp': 55, 'sib': 2, 'Tau': 57, 'po': 9, 'va': 12, 'NaN': 1}}
                                          NaN     Tau     po     pp    sib  \
colum

In [100]:

def chi2_cont_row(df):
    stat, p, dof, expected = chi2_contingency(df, correction=True)
    print('dof=%d' % dof)
    print(expected)

    # interpret test-statistic
    prob = 0.95
    critical = chi2.ppf(prob, dof)
    print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))

    if abs(stat) >= critical:
        print('Dependent (reject H0)')
    else:
        print('Independent (fail to reject H0)')


    # interpret p-value
    alpha = 1.0 - prob
    print('significance=%.3f, p=%.3f' % (alpha, p))
    if p <= alpha:
        print('Dependent (reject H0)')
    else:
        print('Independent (fail to reject H0)')
    return stat, p, dof, expected 

In [134]:
def calcFreqMeasures2DDict(data, cCurrT, columnResponse, cNextT, setResponses):
    F = dict()
    for index, row in data.iterrows():
        currT = row[cCurrT]
        nextT = row[cNextT]
        for cMeasureValue in setResponses:
            values = str(row[columnResponse]).split(';')
            values = [x.strip() for x in values ]
            if(cMeasureValue in values):
                if currT not in F:
                    F[currT] = dict()
                    F[currT]["columnTot"] = dict()
                if cMeasureValue not in F[currT]:
                    F[currT][cMeasureValue] = dict()
                if nextT not in F[currT][cMeasureValue]:
                    if nextT != np.nan:
                        F[currT][cMeasureValue][nextT] = 0
                F[currT][cMeasureValue][nextT] += 1      
        if nextT not in F[currT]["columnTot"]:
            F[currT]["columnTot"][nextT] = 0
        F[currT]["columnTot"][nextT] += 1
        
    return F